In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from tqdm import tqdm
from datetime import datetime
from heapq import heappush,heappop
from collections import Counter,defaultdict
import multiprocessing as mp

PATH="/home/yui/Documents/data/recommender/ratings_Beauty.csv"

func = lambda x:datetime.utcfromtimestamp(x).\
        strftime('%Y-%m-%d %H:%M:%S')
df=pd.read_csv(PATH)
df['Date']=df['Timestamp'].apply(func)
df.head(5)

,UserId,ProductId,Rating,Timestamp,Date
0,A39HTATAQ9V7YF,0205616461,5.0,1369699200,2013-05-28 00:00:00
1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200,2012-12-14 00:00:00
2,A1Z513UWSAAO0F,0558925278,5.0,1404691200,2014-07-07 00:00:00
3,A1WMRR494NWEWV,0733001998,4.0,1382572800,2013-10-24 00:00:00
4,A3IAAVS479H7M7,0737104473,1.0,1274227200,2010-05-19 00:00:00


In [2]:
print("No. of Unique Users: ",df['UserId'].unique().shape)
print("No. of Unique Products: ",df["ProductId"].unique().shape)
print("Shape of dataframe: ",df.shape)

No. of Unique Users:  (1210271,)
No. of Unique Products:  (249274,)
Shape of dataframe:  (2023070, 5)


#### User-User Collaborative Filtering
Given the rating matrix, $i$ is the user, $j$ is the item,
$$
\mathbf{R} = \begin{pmatrix}
r_{00} & \cdots & r_{0j}\\
\vdots & \ddots & \vdots\\
r_{i0} & \cdots & r_{ij}\\
\end{pmatrix}
$$
Originally, the average rating for item j can be treated as suggestion criteria, 
$$s_j = \frac{1}{|\Omega_j|}\sum_{i\in\Omega_j}r_{ij}$$
where $\Omega_j$ set of all users who rated item $j$, $r_{ij}$ rating user $i$ gave item $j$.

Personalized score can be written as, 
$$s_{ij}=\frac{1}{|\Omega_j|}\sum_{i'\in\Omega_j }r_{i'j}$$

But some users have less high ratings than other users, it might be not be too fair to include in the same scale. The better measurement of rating should be average of the deviation between the average ratings of user $\bar{r}_i$ and the rating of the product $r_{ij}$.


$$\begin{align*}
\delta_{ij} &= r_{ij}-\bar{r}_i \\
\hat{\delta}_{ij} &= \frac{1}{|\Omega|}\sum_{i'\in \Omega_j} r_{i'j}-\bar{r}_{i'} \\
s_{ij} &= \bar{r}_i +\hat{\delta}_{ij}
\end{align*}$$

The algorithm is to predictive the empty cell $s_{ij}$ as $\hat{r}_{ij}$ so as to guess what user $i$ might rate item $j$. This becomes a regression problem, and mean-squared error is a good metric to estimate the model performance, 

$$\Delta = \frac{1}{\Omega}\sum_{i,j\in\Omega}(r_{ij}-\hat{r}_{ij})^2$$
where $\Omega$ set of pairs $(i,j)$ where user $i$ has rated item $j$.

The weight ratings can be used to suggest similar users with similar preferences to watch the same movie, but unsuggest different users to watch the same. 

$$s_{ij}=\frac{\sum_{i'\in\Omega_j} w_{ii'}r_{i'j}}{\sum_{i'\in\Omega_j }|w_{ii'}|}$$

where the weight should be great if both users are similar, be small if different.

Finally, the expected rating should be summarized as,

$$s_{ij}=\bar{r}_i + \frac{\sum_{i'\in\Omega_j} w_{ii'}(r_{i'j}-\bar{r}_{i'})}{\sum_{i'\in\Omega_j }|w_{ii'}|}$$

The weight can be calculated as pearson correlation coefficient, 

$$
\rho_{xy} = \frac{\sum^N_{i=1}(x_i-\bar{x})(y_i-\bar{y})}{\sqrt{\sum^N_{i=1}(x_i-\bar{x})^2}\sqrt{\sum^N_{i=1}(y_i-\bar{y})^2}}
$$

$$
w_{ii'} = \frac{\sum_{j\in\Psi_{ii'}}(r_{ij}-\bar{r}_i)(r_{i'j}-\bar{r}_{i'})}{\sqrt{\sum_{j\in\Psi_{ii'}}(r_{ij}-\bar{r}_i)^2}\sqrt{\sum_{j\in\Psi_{ii'}}(r_{i'j}-\bar{r}_{i'})^2}}
$$

where $\Psi_i$ set of items that user $i$ has rated, $\Psi_{ii'}$ set of items both user $i$ and $i'$ have rated, i.e.$\Psi_{ii'}=\Psi_i\cap\Psi_{i'}$. This is equivalent to cosine similarity since $x$ and $y$ are deviations already.

$$\cos\theta = \frac{x^\top y}{|x||y|} = \frac{\sum^N_{i=1}x_iy_i}{\sqrt{\sum^N_{i=1}x_i^2}\sqrt{\sum^N_{i=1}y_i^2}}$$

In [3]:
u2id,id2u,p2id,id2p={},{},{},{}
u2p = defaultdict(dict)
for i in tqdm(range(df.shape[0])):
    user = df.iloc[i]['UserId']
    item = df.iloc[i]['ProductId']
    rating = df.iloc[i]['Rating']
    if user not in u2id:
        u2id[user]=i
        id2u[i]=user
    if item not in p2id:
        p2id[item]=i
        id2p[i]=item
    rateDict = u2p.get(u2id[user],{})
    rateDict[p2id[item]]=rating
    u2p[u2id[user]] = rateDict

100%|██████████| 2023070/2023070 [07:38<00:00, 4408.23it/s]


In [4]:
len(u2p)

1210271

In [5]:
set(u2p[0])

{0, 899062, 969481, 1499663}

In [6]:
def findMatchesItem(target):
    rated = u2p[target]
    p = set(rated)
    if len(p)>3: return 
    res = [] # user who has rated the same products
    for i in range(len(u2p)):
        if i==target:
            continue
        rated_ = u2p[i]
        common = p & set(rated_)
        if len(common)==len(p):
            res.append(i)
    if len(p)>1 and len(res)>0:
        print(target,res)

In [7]:
with mp.Pool(processes=12) as pool:
    with tqdm(total=len(u2p)) as pbar:
        for i, _ in enumerate(pool.imap_unordered(\
            findMatchesItem, range(0,len(u2p)))):
            pbar.update()
            if i==600:
                break

  0%|          | 139/1210271 [00:08<23:39:14, 14.21it/s]

143 [147]


  0%|          | 147/1210271 [00:08<21:56:51, 15.32it/s]

147 [143]


  0%|          | 221/1210271 [00:12<17:07:48, 19.62it/s]

...

  0%|          | 534/1210271 [00:27<16:22:39, 20.52it/s]

528 [498, 517]


  0%|          | 599/1210271 [00:30<13:22:30, 25.12it/s]

591 [606]


  0%|          | 601/1210271 [00:31<17:21:26, 19.36it/s]


In [8]:
for i in [529,506,510,527,532]:
    print(u2p[i])

{494: 5.0, 1080644: 5.0}
{494: 5.0, 1080644: 4.0}
{494: 5.0, 1080644: 5.0}
{494: 5.0, 1080644: 5.0, 1391035: 5.0, 1702927: 5.0}
{494: 5.0, 1080644: 5.0}


#### User-User Collaborative Filtering
Given the rating matrix, $i$ is the user, $j$ is the item,
$$
\mathbf{R} = \begin{pmatrix}
r_{00} & \cdots & r_{0j}\\
\vdots & \ddots & \vdots\\
r_{i0} & \cdots & r_{ij}\\
\end{pmatrix}
$$

The expected rating should be summarized as,

$$s_{ij}=\bar{r}_i + \frac{\sum_{i'\in\Omega_j} w_{ii'}(r_{i'j}-\bar{r}_{i'})}{\sum_{i'\in\Omega_j }|w_{ii'}|}$$

The weight can be calculated as pearson correlation coefficient, 

$$
w_{ii'} = \frac{\sum_{j\in\Psi_{ii'}}(r_{ij}-\bar{r}_i)(r_{i'j}-\bar{r}_{i'})}{\sqrt{\sum_{j\in\Psi_{ii'}}(r_{ij}-\bar{r}_i)^2}\sqrt{\sum_{j\in\Psi_{ii'}}(r_{i'j}-\bar{r}_{i'})^2}}
$$
where $\Psi_i$ set of items that user $i$ has rated, $\Psi_{ii'}$ set of items both user $i$ and $i'$ have rated, i.e.$\Psi_{ii'}=\Psi_i\cap\Psi_{i'}$. 

In [10]:
user = 529
# product = 1391035
product = 1391000
candidates = [506,510,527,532]
u2p[527][product]=4
setProduct = set(u2p[user])
up,down = 0,0
eps = np.finfo(float).eps
user_mean = np.array(list(u2p[user].values())).mean()
for candidate in candidates:
    if product not in u2p[candidate]:
        continue #skip
    common = setProduct & set(u2p[candidate])
    difu = np.array([u2p[user][c] \
            for c in common])-user_mean+eps
    candidate_mean = np.array(list(u2p[candidate].values())).mean()
    difu_ = np.array([u2p[candidate][c] \
            for c in common])-candidate_mean+eps
    w = ((difu*difu_)).sum()/np.sqrt((difu**2).sum())/np.sqrt((difu_**2).sum())
    up += w*(u2p[candidate].get(product,4)-candidate_mean)
    down += w
s = user_mean + up/down
print(up,down,candidate_mean,user_mean,up/down)
print(user_mean,s)

-0.7999999999999998 1.0 4.8 5.0 -0.7999999999999998
5.0 4.2


### Movie recommendation dataset
https://www.kaggle.com/grouplens/movielens-20m-dataset#rating.csv



In [2]:
PATH="/home/yui/Documents/data/recommender/movieLens20M/rating.csv"
df = pd.read_csv(PATH)
df.head(5)

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [3]:
df.iloc[180:190]

,userId,movieId,rating,timestamp
180,2,260,5.0,2000-11-21 15:36:54
181,2,266,5.0,2000-11-21 15:32:28
182,2,469,3.0,2000-11-21 15:29:58
183,2,480,5.0,2000-11-21 15:32:00
184,2,541,5.0,2000-11-21 15:36:54
185,2,589,5.0,2000-11-21 15:30:58
186,2,891,2.0,2000-11-21 15:36:09
187,2,908,4.0,2000-11-21 15:31:31
188,2,924,5.0,2000-11-21 15:36:54
189,2,1121,3.0,2000-11-21 15:29:58


In [4]:
print("Dataframe shape: ",df.shape)
print("Unique User: ",len(df["userId"].unique()))
print("Unique Movie: ",len(df["movieId"].unique()))

Dataframe shape:  (20000263, 4)
Unique User:  138493
Unique Movie:  26744


In [5]:
users = df["userId"].unique()
movies = df["movieId"].unique()
print(users[:5],movies[:5])

[1 2 3 4 5] [ 2 29 32 47 50]


#### Tradeoff
- Due to the massive amount of data we are processing, I choose the user with at least 50 movies rated, the least number of common movies rated between the user being suggested and the user as the guide is 25, and need at least 25 users.
- To avoid only suggesting only by first n users, it is possible to shuffle according to userId.


In [6]:
def findNMatches(df,targetUser,leastCommon=25,minMoviesWatched=50,noRef=25):
    tardf = df[df["userId"]==targetUser]
    u2m = defaultdict(dict)
    valCountsUserId = df["userId"].value_counts()

    for j in range(tardf.shape[0]):
        movieId = tardf.iloc[j]["movieId"]
        u2m[targetUser][movieId] = tardf.iloc[j]["rating"]
    avg = np.array(list(u2m[targetUser].values())).mean()
    u2m[targetUser] = (u2m[targetUser],avg)
    tarSet = set(u2m[targetUser][0])
    # print(tarSet)
    prevId,storeSet,storeDict = 0,set(),dict()
    for i in tqdm(range(df.shape[0])):
        row = df.iloc[i]
        userId = row["userId"]
        if valCountsUserId[userId]<minMoviesWatched:
            continue
        if userId==targetUser:
            continue
        if userId!=prevId:
            if len(tarSet&storeSet)>=leastCommon:
                avg = np.array(list(storeDict.values())).mean()
                u2m[prevId]=(storeDict,avg)
            storeSet,storeDict=set(),dict()
        prevId = userId
        storeSet.add(row["movieId"])
        storeDict[row["movieId"]]=row["rating"]
        if len(u2m)>noRef:
            break
    return u2m

#### User-User Collaborative Filtering
Given the rating matrix, $i$ is the user, $j$ is the item,
$$
\mathbf{R} = \begin{pmatrix}
r_{00} & \cdots & r_{0j}\\
\vdots & \ddots & \vdots\\
r_{i0} & \cdots & r_{ij}\\
\end{pmatrix}
$$

The expected rating should be summarized as,

$$s_{ij}=\bar{r}_i + \frac{\sum_{i'\in\Omega_j} w_{ii'}(r_{i'j}-\bar{r}_{i'})}{\sum_{i'\in\Omega_j }|w_{ii'}|}$$

The weight can be calculated as pearson correlation coefficient, 

$$
w_{ii'} = \frac{\sum_{j\in\Psi_{ii'}}(r_{ij}-\bar{r}_i)(r_{i'j}-\bar{r}_{i'})}{\sqrt{\sum_{j\in\Psi_{ii'}}(r_{ij}-\bar{r}_i)^2}\sqrt{\sum_{j\in\Psi_{ii'}}(r_{i'j}-\bar{r}_{i'})^2}}
$$
where $\Psi_i$ set of items that user $i$ has rated, $\Psi_{ii'}$ set of items both user $i$ and $i'$ have rated, i.e.$\Psi_{ii'}=\Psi_i\cap\Psi_{i'}$. 

In [7]:
def computeW(dict1,dict2,m1,m2):
#     eps = np.finfo(float).eps
    eps = 0
    common = set(dict1)&set(dict2)
    difu = np.array([dict1[c] for c in common])-m1+eps
    difu_ = np.array([dict2[c] for c in common])-m2+eps
    up = ((difu*difu_)).sum()
    down = np.sqrt((difu**2).sum())*np.sqrt((difu_**2).sum())
    return up/down,set(dict1)^set(dict2)

In [8]:
def suggest(u2m,targetUser):
    items = defaultdict(dict)
    # key(movieid) -> up:0,down:0
    targetDict,targetM = u2m[targetUser]
    for candidate in u2m:
        if candidate==targetUser:
            continue
        tmpDict,tmpM = u2m[candidate]
        w,xorSet = computeW(targetDict,tmpDict,targetM,tmpM)
        for item in xorSet:
            if item in targetDict:
                continue
            up,down = items.get(item,(0,0))
#             print(tmpDict[item]-tmpM,w)
            up += w*(tmpDict[item]-tmpM)
            down += abs(w)
            items[item]=(up,down)
    suggestions = {}
    for item in items:
        up,down = items[item]
#         print(item,up,down,up/down)
        suggestions[item]=up/down
    return suggestions


In [9]:
u2m = findNMatches(df,1)
print(len(u2m))
suggestions = suggest(u2m,1)
sortSuggests = sorted(suggestions.items(),
        key=lambda x:-x[1])
# print(sortSuggests)

  0%|          | 15260/20000263 [00:01<33:45, 9868.24it/s] 

26


In [10]:
print(sortSuggests[0],sortSuggests[-1])

(1054, 2.705521472392638) (44828, -3.445436507936508)


In [11]:
print(np.array(list(u2m[1][0].values())).mean())

3.742857142857143
